In [1]:
#Importing libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from wordcloud import WordCloud,STOPWORDS
import sqlite3

In [2]:
import requests

In [3]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('liSqdZAKoID2d-bIQH5M9A', 'tecIJkvcWwnQK9exC0fHNCnudtdYVA')

In [4]:
# here we pass our login method (password), username, and password
data = {'grant_type': 'password'}

In [6]:
with open('pw.txt', 'r') as file:
# Step 2: Read the contents of the file
        lines = file.readlines()

        # Iterate over each line
        for line in lines:
                # Split the line into username and password
                username, password = line.strip().split(':')

                # Store the username and password in the dictionary
                data['username'] = username
                data['password'] = password

In [7]:
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MySideProject/0.0.1'}

In [8]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

In [9]:
# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

In [10]:
# add authorization to our headers dictionary
headers['Authorization']=f'bearer {TOKEN}'

In [11]:
def df_from_response(res):
    # initialize temp list to store dictionaries
    data_list = []

    # loop through each post pulled from res and append to data_list
    for post in res.json()['data']['children']:
        data_list.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        })

    return data_list

Subreddits to explore

- politics
- worldnews
- news
- energy
- environment
- business
- worldevents

In [12]:
# initialize dataframe and parameters for pulling data in loop
data = pd.DataFrame()
params = {'limit': 100}

# loop through 10 times (returning 1K posts)
for i in range(10):
    # make request
    res = requests.get("https://oauth.reddit.com/r/politics/new",
                       headers=headers,
                       params=params)

    # get list of dictionaries from response
    data_list = df_from_response(res)

    # convert list of dictionaries to DataFrame and append to data
    data = pd.concat([data, pd.DataFrame(data_list)], ignore_index=True)

    # take the final row (oldest entry)
    row = data_list[-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname

In [13]:
data.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score,link_flair_css_class,created_utc,id,kind
0,politics,Trump Jokes About How Longtime Aide Could End ...,,1.00,3,0,3,None,2024-04-12T09:54:31Z,1c23gnn,t3
1,politics,US to probe if Chinese cars pose national data...,,1.00,4,0,4,None,2024-04-12T09:41:11Z,1c23a05,t3
2,politics,Irish taoiseach and Spanish PM to discuss Pale...,,0.54,1,0,1,None,2024-04-12T09:09:41Z,1c22tft,t3
3,politics,Why Trump’s ‘hush money’ case is bigger than h...,,0.94,15,0,15,None,2024-04-12T09:01:51Z,1c22p5v,t3
4,politics,1864 AZ Speaker of the house who helped pass t...,,0.86,35,0,35,None,2024-04-12T08:20:41Z,1c221tb,t3


In [15]:
# Connect to SQLite database (create if it doesn't exist)
conn = sqlite3.connect('reddit_data.db')

In [16]:
# Create a cursor object to execute SQL commands
cursor = conn.cursor()

In [17]:
# Create a table to store your data
cursor.execute('''CREATE TABLE IF NOT EXISTS reddit_posts (
                    id INTEGER PRIMARY KEY,
                    subreddit TEXT,
                    title TEXT,
                    selftext TEXT,
                    upvote_ratio REAL,
                    ups INTEGER,
                    downs INTEGER,
                    score INTEGER,
                    link_flair_css_class TEXT,
                    created_utc INTEGER,
                    kind TEXT,
                    cleaned TEXT
                )''')

In [18]:
# Insert data into the table
for index, row in data.iterrows():
    cursor.execute('''INSERT INTO reddit_posts
                      (subreddit, title, selftext, upvote_ratio, ups, downs, score,
                       link_flair_css_class, created_utc, kind)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                   (row['subreddit'], row['title'], row['selftext'], row['upvote_ratio'],
                    row['ups'], row['downs'], row['score'], row['link_flair_css_class'],
                    row['created_utc'], row['kind']))

# Commit changes and close connection
conn.commit()
conn.close()